In [7]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS


In [8]:
# --- STEP 1: Load PDF and Prepare Chunks ---
loader = PyPDFLoader("./../data/wikipedia_naruto.pdf")  # Replace with your own PDF file
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [9]:
from dotenv import load_dotenv
import os

# Load variables from .env into environment
load_dotenv()

# Retrieve the key
LLM_API_KEY = os.getenv("GOOGLE_API_KEY")

print("Loaded API Key:", LLM_API_KEY is not None)

Loaded API Key: True


In [10]:
# --- STEP 2: Create Vector Store using Gemini Embeddings ---
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=LLM_API_KEY)
vectorstore = FAISS.from_documents(chunks, embeddings)

In [11]:
# --- STEP 3: Create a Search Tool ---
@tool
def search_pdf(query: str) -> str:
    """Search the PDF for relevant information and return the best matching content."""
    results = vectorstore.similarity_search(query, k=3)
    return "\n\n".join([doc.page_content for doc in results])

In [ ]:
# --- STEP 4: Setup Gemini LLM ---
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=LLM_API_KEY
)


In [12]:
# --- STEP 5: Create Agent Prompt ---
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the tools to search the PDF and summarize answers."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])


In [13]:
# --- STEP 6: Create Agent and Executor ---
tools = [search_pdf]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
# --- STEP 7: Run Agent ---

question = "Summarize the key points about climate change from the document."
result = agent_executor.invoke({"input": question})
print("\nFINAL ANSWER:\n", result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_pdf` with `{'query': 'climate change key points'}`


becomes the Sixth Hokage and pardons Sasuke for his crimes. Years later, Kakashi steps down while 
Naruto marries Hinata Hyuga and becomes the Seventh Hokage, raising the next generation.

Source: https://en.wikipedia.org/wiki/Naruto   
    
 This article is about the manga series. For the anime, see Naruto (TV series). For the title character, see 
Naruto Uzumaki. For other uses, see Naruto (disambiguation).   
 Not to be confused with Naruhito, the emperor of Japan.   
 Naruto   
    
 Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of 
Naruto Uzumaki, a young ninja who seeks recognition from his peers and dreams of becoming the 
Hokage, the leader of his village. The story is told in two parts: the first is set in Naruto's pre-teen years 
(volumes 1±27), and the second in his teens (volumes 28±72). The series is based on t